In [1]:
import numpy as np
import seaborn as sns
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
from tqdm import tqdm
from small2DNet import small2DNet
from util import add_color, colorize, colorize_gaussian, calculate_correct_loss
from colorMNist import colorMNist
import pickle

In [2]:
# MNIST DATASET

# Load data from pickle file
cmnist_train, cmnist_val, cmnist_test = pickle.load(open("custom_datasets/5k/cmnist_gaussian_24.pkl", "rb"))
print(len(cmnist_train), len(cmnist_val), len(cmnist_train) + len(cmnist_val))

# Create datasets
train_dataset = colorMNist(cmnist_train)
val_dataset = colorMNist(cmnist_val)
test_dataset = colorMNist(cmnist_test)

# Dataloaders
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32,
                                               shuffle=True, num_workers = 0)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32,
                                               shuffle=True, num_workers = 0)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32,
                                               shuffle=False, num_workers = 0)

4000 1000 5000


In [3]:
# Layers of the model
model_layers = [28, "M", 55, "M", 111,"M"]
# model_layers = [16, "M", 96, "M", 64,"M"]
# Set seed
torch.manual_seed(12)
# Create model
linear_neurons = 512
model = small2DNet(model_layers, model_layers[-2], linear_neurons)
# Load file and save file
# lfile = "Gaussian2D_12"
sfile = "test_1"
# Load model
# model.load_state_dict(torch.load('model_saves/new_fair/'+ lfile + '.pth'))
# Put model on gpu
model.cuda()
# Loss function
loss_fn = torch.nn.CrossEntropyLoss()
# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [4]:
# Number of epochs to train
epochs = 20

# Placeholder variables to put training and validation accuracies and losses per epoch
train_accuracies = []
train_losses = []
val_accuracies = []
val_losses = []

for epoch in tqdm(range(epochs), total=epochs, desc='Training'):
    # print("Epoch", epoch + 1, "/", epochs)
    
    # Update learning rate
    # if (epoch + 1) % 20 == 0:
    #     for g in optimizer.param_groups:
    #         g['lr'] /= 10
    
    # Put model on training mode
    model.train()
    train_total_correct = 0
    train_total_loss = []
    
    for (images, labels) in train_dataloader:
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Calculate number correct and loss in batch
        correct, loss = calculate_correct_loss(model, loss_fn, images, labels)
        
        # print(y.shape)
        # print(y)
        # print(y[:][:120])
        # y = torch.narrow(y, 1, 0, 120)
        # print(y.shape)
        # Backpropagation
        loss.backward()
        # Step function
        optimizer.step()
        
        # Update amount correct and loss with current batch
        train_total_correct += correct
        train_total_loss.append(loss.item())
#         break
        
#     break
        
    # Append epoch accuracy and loss
    train_accuracies.append(train_total_correct / len(train_dataset))
    train_losses.append(sum(train_total_loss) / len(train_total_loss))
    
    # Put model on evaluation mode
    model.eval()
    val_total_correct = 0
    val_total_loss = []
    
    # Without gradient calculation
    with torch.no_grad():
        for (images, labels) in val_dataloader:
        
            # Calculate number correct and loss in batch
            correct, loss = calculate_correct_loss(model, loss_fn, images, labels)

            # Update amount correct and loss with current batch
            val_total_correct += correct
            val_total_loss.append(loss.item())

    # Append epoch accuracy and loss
    val_accuracies.append(val_total_correct / len(val_dataset))
    val_losses.append(sum(val_total_loss) / len(val_total_loss))
    

# Print accuracies and losses per epoch
for i in range(epochs):
    print("Epoch", i + 1)
    print("Train acc and loss\t", train_accuracies[i], "\t", train_losses[i])
    print("Val acc and loss\t", val_accuracies[i], "\t", val_losses[i])

Training: 100%|██████████| 20/20 [00:11<00:00,  1.69it/s]

Epoch 1
Train acc and loss	 0.34375 	 1.9183515758514404
Val acc and loss	 0.587 	 1.1486585587263107
Epoch 2
Train acc and loss	 0.61775 	 0.968812864780426
Val acc and loss	 0.682 	 0.8316646795719862
Epoch 3
Train acc and loss	 0.727 	 0.7479231700897216
Val acc and loss	 0.813 	 0.5903870705515146
Epoch 4
Train acc and loss	 0.82275 	 0.51640937936306
Val acc and loss	 0.822 	 0.5472659189254045
Epoch 5
Train acc and loss	 0.87575 	 0.3729446861743927
Val acc and loss	 0.886 	 0.4144955761730671
Epoch 6
Train acc and loss	 0.90525 	 0.30044219422340396
Val acc and loss	 0.911 	 0.2913072123192251
Epoch 7
Train acc and loss	 0.915 	 0.2642312902212143
Val acc and loss	 0.912 	 0.28073985676746815
Epoch 8
Train acc and loss	 0.928 	 0.2220593138486147
Val acc and loss	 0.944 	 0.20308741240296513
Epoch 9
Train acc and loss	 0.932 	 0.2081288343667984
Val acc and loss	 0.938 	 0.20129753509536386
Epoch 10
Train acc and loss	 0.938 	 0.1832239996343851
Val acc and loss	 0.941 	 0.19355

In [10]:
# Save the model
sfile = "test_1"
torch.save(model.state_dict(), 'testsave/' + sfile + '.pth')

In [11]:
# Layers of the model
model_layers = [28, "M", 55, "M", 111,"M"]
# model_layers = [16, "M", 96, "M", 64,"M"]
# Create model
model = small2DNet(model_layers, model_layers[-2], linear_neurons)
# Load model
model.load_state_dict(torch.load('testsave/'+ sfile + '.pth'))
# Put model on gpu
model.cuda()
hi = 5

In [12]:
# Testing
    
wrong_dict = {}
right_dict = {}

for i in range(10):
    wrong_dict[i] = {}
    for j in range(10):
        wrong_dict[i][j] = 0
    right_dict[i] = 0
    
for it in range(1):
    # Total and amount correct
    test_correct = 0
    test_total = 0

    # File to write predicted labels
    # with open("predicted_labels.txt", "w") as f:
    #     f.write("\n")


    # Put the model in evaluation mode
    model.eval()

    # Without gradient calculation
    with torch.no_grad():
        for (images, labels) in tqdm(test_dataloader):

            # Put images
            images = images.cuda()

            # Predicted labels
            preds = model(images)

            # Top predictions per image
            _, top_preds = torch.max(preds, 1)

            # Predictions and images back on cpu
            top_preds = top_preds.cpu()
            images = images.cpu()
            
            # Check the predicted
            for i in range(len(labels)):
                if top_preds[i].item() == labels[i].item():
                    right_dict[top_preds[i].item()] += 1
                else:
                    wrong_dict[labels[i].item()][top_preds[i].item()] += 1

            # Amount of correct predictions
            predictions = [top_preds[i].item() == labels[i].item() for i in range(len(labels))]
            correct = np.sum(predictions)

    #         # Show batch images
    #         fig, axs = plt.subplots(4,8, figsize=(28, 28), facecolor='w', edgecolor='k')
    #         fig.subplots_adjust(hspace = .5, wspace=.001)
    #         axs = axs.ravel()
    #         for i in range(len(images)):
    #             axs[i].imshow(images[i].permute(1, 2, 0))

    #         break

            # Update total correct and total images
            test_correct += correct
            test_total += len(images)


    print("Correct", test_correct, "/", test_total, "Accuracy:", test_correct / test_total)

100%|██████████| 313/313 [00:00<00:00, 383.31it/s]

Correct 9655 / 10000 Accuracy: 0.9655
